***CLEANING UP DIAGNOSIS AND SYMPTOMS DATASETS***

In [1208]:
import pandas as pd

dis_ids = pd.read_csv('diagnosisNamenId.csv')
symp_ids = pd.read_csv('symptomsNids.csv')
syms_dis_weights = pd.read_csv('symptomsDiseasesNWeight.csv')
diagnosis_amount_df = pd.read_csv("symptomsExtraInfo.csv")

In [1209]:
import numpy as np
from sklearn.naive_bayes import BernoulliNB

In [1210]:
# print(dis_ids.info())
# print(symp_ids.info())
# print(syms_dis_weights.info())
diagnosis_amount_df

,_id,chief_complaint_id,chief_complaint_name_id,name,reason_code,life_threatening,msp_enabled,primary_complaint,diagnosis_count
0,1,1,1,Upper abdominal pain,15453.0,0.0,1.0,1.0,44
1,2,1,2,Upper belly pain,15453.0,0.0,1.0,NaN,44
2,4,2,4,Lower abdominal pain,15452.0,0.0,1.0,1.0,48
3,5,2,5,Lower belly pain,15452.0,0.0,1.0,NaN,48
4,6,3,6,Abscess (Collection of pus),NaN,NaN,NaN,NaN,17
...,...,...,...,...,...,...,...,...,...
384,731,303,503,Arm stiffness or tightness,19453.0,0.0,0.0,1.0,14
385,733,116,505,Lack of energy,10150.0,0.0,0.0,0.0,50
386,737,200,509,Trouble seeing,NaN,NaN,NaN,0.0,9
387,738,304,510,High blood pressure,NaN,NaN,NaN,1.0,30


In [1211]:
# for each diagnosis have entry of symptom ids
# new columns = diagnosis, symptom ids, weight of disease !!!!!!!!!!!!!!!!
# check how mnay values of weights assigned to each diagnosis
final_entries = []
# for each disease id get entries of symptoms n weight
for i,x in enumerate(dis_ids['id'].unique()):
    symtom_ids = syms_dis_weights[syms_dis_weights.did.isin([x])]['syd'].tolist()
    weights = syms_dis_weights[syms_dis_weights.did.isin([x])]['wei'].tolist()
    if len(weights)>0:
        weight_av = sum(weights)/len(weights)
    final_entries.append([x, symtom_ids, weight_av])

final_df = pd.DataFrame(final_entries)
final_df.head(5)

,0,1,2
0,2.0,"[99.0, 284.0]",0.500000
1,3.0,"[102.0, 2.0]",2.000000
2,4.0,"[91.0, 152.0, 182.0, 6.0, 11.0, 13.0, 23.0, 39...",0.357143
3,5.0,"[17.0, 126.0, 159.0, 194.0, 195.0]",1.000000
4,6.0,"[71.0, 77.0, 149.0, 153.0]",3.000000


In [1212]:
final_df.columns = ['Diagnosis', 'Symptoms', 'Weight']

In [1213]:
# Putting words instead of ids

for i,x in enumerate(final_df['Diagnosis']):
    final_df['Diagnosis'].loc[i] = dis_ids.loc[(dis_ids['id'] == x)]['title'].values

In [1214]:
final_df['Symptoms']
final_df[final_df.Symptoms == "e"]

,Diagnosis,Symptoms,Weight


In [1215]:
for i,x in enumerate(final_df['Symptoms']):
    sympnames = []
    for sympid in x:
        if len(symp_ids.loc[(symp_ids['_id'] == sympid)]['symptom'].tolist())>0:
            sympnames.append(symp_ids.loc[(symp_ids['_id'] == sympid)]['symptom'].tolist()[0])
   # print(sympnames)
    final_df['Symptoms'].loc[i] = sympnames
    

final_df.head()

,Diagnosis,Symptoms,Weight
0,[Abdominal swelling],[],0.500000
1,[Abdominal trauma],"[Trauma, Lower abdominal pain]",2.000000
2,[Abrasions (scrapes)],"[Painful rash, Arm ache or pain, Pain or soren...",0.357143
3,[ACE inhibitor induced cough blood pressure me...,"[Cough, Mouth swelling, Lip swelling, Eye swel...",1.000000
4,[acetaminophen overdose Adverse reaction to ac...,"[Overdose, Poisoning]",3.000000


In [1216]:
final_df['Weight'] = final_df['Weight'].round(2)
final_df.head()

,Diagnosis,Symptoms,Weight
0,[Abdominal swelling],[],0.50
1,[Abdominal trauma],"[Trauma, Lower abdominal pain]",2.00
2,[Abrasions (scrapes)],"[Painful rash, Arm ache or pain, Pain or soren...",0.36
3,[ACE inhibitor induced cough blood pressure me...,"[Cough, Mouth swelling, Lip swelling, Eye swel...",1.00
4,[acetaminophen overdose Adverse reaction to ac...,"[Overdose, Poisoning]",3.00


In [1217]:
final_df.head()
final_df.loc[final_df['Diagnosis'] == "Premature ejaculation"]

,Diagnosis,Symptoms,Weight
1312,[Premature ejaculation],[],0.25


In [1218]:
# final_df = final_df[final_df.astype(str)['Symptoms'] != '[]']

In [1219]:
final_df.loc[final_df['Diagnosis'] == "Premature ejaculation"]

,Diagnosis,Symptoms,Weight
1312,[Premature ejaculation],[],0.25


In [1220]:
final_df

,Diagnosis,Symptoms,Weight
0,[Abdominal swelling],[],0.50
1,[Abdominal trauma],"[Trauma, Lower abdominal pain]",2.00
2,[Abrasions (scrapes)],"[Painful rash, Arm ache or pain, Pain or soren...",0.36
3,[ACE inhibitor induced cough blood pressure me...,"[Cough, Mouth swelling, Lip swelling, Eye swel...",1.00
4,[acetaminophen overdose Adverse reaction to ac...,"[Overdose, Poisoning]",3.00
...,...,...,...
1308,[Hypersensitivity pneumonitis allergic lung di...,[],0.00
1309,[Congestive heart failure CHF],"[Cough, Lethargy (Sluggishness), Shortness of ...",1.50
1310,"[Coronary artery disease CAD, heart disease]","[Chest pressure, Chest pain, Dizziness, Shortn...",1.33
1311,[Essential hypertension high blood pressure],"[Chest pain, Chest pressure, Headache]",0.25


In [1221]:
# for k,x in enumerate(final_df['Diagnosis']):
#     new_list = []
#     for word in x:
#         new_list.append(''.join([i if ord(i) < 128 else ' ' for i in word]))
# #         print(''.join([i if ord(i) < 128 else ' ' for i in word]))
# #     print(new_list)
#     final_df['Diagnosis'].loc[k] = new_list
# final_df

In [1222]:
#for x in final_df['Diagnosis'].head():

def cleanString(s):
    for i,l in enumerate(s):
        if (l.isalpha()==False):
#             print(l)
            return s[:i]
        

In [1223]:
# final_df['Diagnosis'] = final_df['Diagnosis'].apply(lambda x: ''.join([" " if ord(i) < 32 or ord(i) > 126 else i for i in x]))
# final_df

In [1224]:
#final_df['Diagnosis'] = final_df['Diagnosis'].apply(cleanString)

# delete all nulls!!!

for i,x in enumerate(final_df['Diagnosis']):
    if len(x)>0:
        wds = x[0].split()
        seperator = " "
        clean = seperator.join(wds)
#         print(clean)
    final_df['Diagnosis'].loc[i] = clean
#     print(clean)

for x in final_df['Symptoms']:
    if len(x)>0:
        for w in x:
            wds = w.split()
            seperator = " "
            clean = seperator.join(wds)
    final_df['Symptoms'].loc[i] = clean


In [1228]:
final_df = final_df[final_df['Diagnosis'] != "Premature ejaculation"]
final_df = final_df[final_df.astype(str)['Symptoms'] != '[]']
final_df

,Diagnosis,Symptoms,Weight
1,Abdominal trauma,"[Trauma, Lower abdominal pain]",2.00
2,Abrasions (scrapes),"[Painful rash, Arm ache or pain, Pain or soren...",0.36
3,ACE inhibitor induced cough blood pressure med...,"[Cough, Mouth swelling, Lip swelling, Eye swel...",1.00
4,acetaminophen overdose Adverse reaction to ace...,"[Overdose, Poisoning]",3.00
7,Acid LSD abuse,[Substance abuse (Drug abuse)],NaN
...,...,...,...
1306,West Nile virus infection mosquito-transmitted...,"[Fever in the returning traveler, Numbness, Se...",1.56
1307,Atrial fibrillation irregular heart rate,"[Dizziness, Heart pulsations and palpitations,...",1.33
1309,Congestive heart failure CHF,"[Cough, Lethargy (Sluggishness), Shortness of ...",1.50
1310,"Coronary artery disease CAD, heart disease","[Chest pressure, Chest pain, Dizziness, Shortn...",1.33


In [1229]:
final_df['Symptoms'].loc[3]
# remove all empty symptoms

['Cough', 'Mouth swelling', 'Lip swelling', 'Eye swelling']

In [1230]:
# final_df.columns

# final_df = final_df[final_df.astype(str)['Symptoms'] != '[]']
# final_df = final_df[final_df.astype(str)['Symptoms'] != '[]']

In [1231]:
final_df

,Diagnosis,Symptoms,Weight
1,Abdominal trauma,"[Trauma, Lower abdominal pain]",2.00
2,Abrasions (scrapes),"[Painful rash, Arm ache or pain, Pain or soren...",0.36
3,ACE inhibitor induced cough blood pressure med...,"[Cough, Mouth swelling, Lip swelling, Eye swel...",1.00
4,acetaminophen overdose Adverse reaction to ace...,"[Overdose, Poisoning]",3.00
7,Acid LSD abuse,[Substance abuse (Drug abuse)],NaN
...,...,...,...
1306,West Nile virus infection mosquito-transmitted...,"[Fever in the returning traveler, Numbness, Se...",1.56
1307,Atrial fibrillation irregular heart rate,"[Dizziness, Heart pulsations and palpitations,...",1.33
1309,Congestive heart failure CHF,"[Cough, Lethargy (Sluggishness), Shortness of ...",1.50
1310,"Coronary artery disease CAD, heart disease","[Chest pressure, Chest pain, Dizziness, Shortn...",1.33


In [1232]:
final_df.to_csv('data.csv')

In [1233]:
diagnosis_amount_df.head(5)

,_id,chief_complaint_id,chief_complaint_name_id,name,reason_code,life_threatening,msp_enabled,primary_complaint,diagnosis_count
0,1,1,1,Upper abdominal pain,15453.0,0.0,1.0,1.0,44
1,2,1,2,Upper belly pain,15453.0,0.0,1.0,NaN,44
2,4,2,4,Lower abdominal pain,15452.0,0.0,1.0,1.0,48
3,5,2,5,Lower belly pain,15452.0,0.0,1.0,NaN,48
4,6,3,6,Abscess (Collection of pus),NaN,NaN,NaN,NaN,17


In [1234]:
final_df.Diagnosis

1                                        Abdominal trauma
2                                     Abrasions (scrapes)
3       ACE inhibitor induced cough blood pressure med...
4       acetaminophen overdose Adverse reaction to ace...
7                                          Acid LSD abuse
                              ...                        
1306    West Nile virus infection mosquito-transmitted...
1307             Atrial fibrillation irregular heart rate
1309                         Congestive heart failure CHF
1310           Coronary artery disease CAD, heart disease
1311           Essential hypertension high blood pressure
Name: Diagnosis, Length: 934, dtype: object

In [1235]:
# Add a column that is the amount of times the diagnosis has been diagnosed
dataframe_james = final_df.copy()

In [1236]:
diagnosis_amount = diagnosis_amount_df[["name","diagnosis_count"]]
diagnosis_amount.name

0             Upper abdominal pain
1                 Upper belly pain
2             Lower abdominal pain
3                 Lower belly pain
4      Abscess (Collection of pus)
                  ...             
384     Arm stiffness or tightness
385                 Lack of energy
386                 Trouble seeing
387            High blood pressure
388               High blood sugar
Name: name, Length: 389, dtype: object

In [1237]:
# create a dataframe with each column a symptoms, diagnosis, weight, count
symptoms = final_df.Symptoms
symptoms

1                          [Trauma, Lower abdominal pain]
2       [Painful rash, Arm ache or pain, Pain or soren...
3       [Cough, Mouth swelling, Lip swelling, Eye swel...
4                                   [Overdose, Poisoning]
7                          [Substance abuse (Drug abuse)]
                              ...                        
1306    [Fever in the returning traveler, Numbness, Se...
1307    [Dizziness, Heart pulsations and palpitations,...
1309    [Cough, Lethargy (Sluggishness), Shortness of ...
1310    [Chest pressure, Chest pain, Dizziness, Shortn...
1311               [Chest pain, Chest pressure, Headache]
Name: Symptoms, Length: 934, dtype: object

In [1238]:
symptoms_set = set(x for l in symptoms for x in l)
symptoms_list = list(symptoms_set)
np.array(symptoms_list).shape

(131,)

In [1239]:
# need to look through final_df and see if symptom appears for diagnosis
symptoms_dataframe = pd.DataFrame(columns=symptoms_list)
symptoms_dataframe

,Sinus pain and pressure,Bleeding tendency,Tremors,Fever of unknown origin,Pain in the ear,Arm ache or pain,Arm numbness (paresthesias),Neck ache or pain,Impotence,Failure to thrive,...,Skin sores,Hand itching,Speech problem,Armpit pain,Ear swelling,Dizziness,Headache,Vaginal bleeding,Depressed,Lethargy (Sluggishness)


In [1240]:
symptoms_dataframe.columns

Index(['Sinus pain and pressure', 'Bleeding tendency', 'Tremors',
       'Fever of unknown origin', 'Pain in the ear', 'Arm ache or pain',
       'Arm numbness (paresthesias)', 'Neck ache or pain', 'Impotence',
       'Failure to thrive',
       ...
       'Skin sores', 'Hand itching', 'Speech problem', 'Armpit pain',
       'Ear swelling', 'Dizziness', 'Headache', 'Vaginal bleeding',
       'Depressed', 'Lethargy (Sluggishness)'],
      dtype='object', length=131)

In [1241]:
symptoms_dataframe.columns.shape

(131,)

In [1244]:
symptoms = final_df.Symptoms
type(symptoms)

symptoms_set = set(x for l in symptoms for x in l)
symptoms_list = list(symptoms_set)
symptoms_list[0:3]

symptoms_dataframe = pd.DataFrame(columns=symptoms_list)
counter = 0
for diagnosis in final_df.Diagnosis:
    symptoms = final_df.loc[final_df['Diagnosis'] == diagnosis].Symptoms
    symptoms_set = set(x for l in symptoms for x in l)
    symptoms_temp_list = list(symptoms_set)
    sym_num_list = list()
    for symptom in symptoms_dataframe.columns:
        if (symptom in symptoms_temp_list):
            sym_num_list.append(1)
        else:
            sym_num_list.append(0)
        
    # add list to dataframe
    symptoms_dataframe.loc[counter] = (sym_num_list)
    counter += 1
#     print("Diagnosis: {}, symptoms {}".format(diagnosis, symptoms_temp_list, sym_num_list))

In [1245]:
columns = [["A","B"]]
test_df = pd.DataFrame(columns=columns)
l = [1,2]
test_df.loc[0] = l 
test_df

,A,B
0,1,2


In [1246]:
frames = [symptoms_dataframe, final_df.Diagnosis, final_df.Weight]

In [1247]:
symptoms_dataframe["Diagnosis"] = final_df.Diagnosis
symptoms_dataframe["Weight"] = final_df.Weight

In [1248]:
symptoms_dataframe.loc[symptoms_dataframe['Diagnosis'] == "Abdominal trauma"]

,Sinus pain and pressure,Bleeding tendency,Tremors,Fever of unknown origin,Pain in the ear,Arm ache or pain,Arm numbness (paresthesias),Neck ache or pain,Impotence,Failure to thrive,...,Speech problem,Armpit pain,Ear swelling,Dizziness,Headache,Vaginal bleeding,Depressed,Lethargy (Sluggishness),Diagnosis,Weight
1,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,Abdominal trauma,2.0


In [1249]:
symptoms_dataframe = symptoms_dataframe.dropna()
symptoms_dataframe

,Sinus pain and pressure,Bleeding tendency,Tremors,Fever of unknown origin,Pain in the ear,Arm ache or pain,Arm numbness (paresthesias),Neck ache or pain,Impotence,Failure to thrive,...,Speech problem,Armpit pain,Ear swelling,Dizziness,Headache,Vaginal bleeding,Depressed,Lethargy (Sluggishness),Diagnosis,Weight
1,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,Abdominal trauma,2.00
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Abrasions (scrapes),0.36
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,ACE inhibitor induced cough blood pressure med...,1.00
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,acetaminophen overdose Adverse reaction to ace...,3.00
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Adenoiditis (a type of lymph node inflammation),1.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
927,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,1,Miller Fisher Syndrome rare nerve disease,0.00
929,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,Burns,2.45
930,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,Fat embolism blood clot from fat,0.00
931,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,"Lupus systemic lupus erythematosus, SLE",0.00


In [1250]:
from sklearn.model_selection import train_test_split
# print(symptoms_dataframe.columns)
X = symptoms_dataframe.copy().drop(columns=["Diagnosis","Weight"])
y = symptoms_dataframe.copy().Diagnosis
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [1251]:
from sklearn.naive_bayes import BernoulliNB
clf = BernoulliNB()
clf.fit(X_train, y_train)
print("Classification on Training Data: {:.3f}".format(clf.score(X_train,y_train)))
print("Classification on Testing Data: {:.3f}".format(clf.score(X_test,y_test)))

Classification on Training Data: 0.805
Classification on Testing Data: 0.000


In [1267]:
from scipy.spatial import distance
def get_similarity(symptoms_vector):
    similarity_df = pd.DataFrame(columns=["Diagnosis","Similarity"])
    counter = 0
    for index, row in symptoms_dataframe.iterrows():
        row = row[:-2]
        similarity = distance.hamming(row,symptoms_vector)
        #similarity_df.iloc[index] = [symptoms_dataframe.Diagnosis[index], similarity]
        similarity_df.loc[index] = [symptoms_dataframe.Diagnosis[index], similarity]
        counter += 1
    return similarity_df 
symptom_vector = symptoms_dataframe.iloc[4][:-2]
a = (symptoms_dataframe.iloc[4][:-2].values)
b = (symptoms_dataframe.columns[:-2].values)
c = list(zip(a, b))
sim_df = get_similarity(symptom_vector)
# sim_df
sim_df = sim_df.sort_values(by="Similarity", ascending=False)
# sim_df.loc[sim_df['Similarity'].idxmin()]
# sim_df
symptoms_dataframe.iloc[4]

Sinus pain and pressure                                                  0
Bleeding tendency                                                        0
Tremors                                                                  0
Fever of unknown origin                                                  0
Pain in the ear                                                          0
                                                ...                       
Vaginal bleeding                                                         0
Depressed                                                                0
Lethargy (Sluggishness)                                                  0
Diagnosis                  Adenoiditis (a type of lymph node inflammation)
Weight                                                                1.67
Name: 9, Length: 133, dtype: object

#### 

In [1253]:
sim_df.loc[sim_df['Similarity'].idxmin()]
    

Diagnosis     Abdominal trauma
Similarity                   0
Name: 1, dtype: object